# Week 12 Handson - Assignment/Project 2: Regression #02

As the previous assignment, the objective of this project is to explore your capability as data analyst. You are challenged to implement data mining methods, given a raw dataset. Then, you need to propose several regression methods for the data, analyze and evaluate it. 

Basic tasks you need to perform in this project: (Note: you can add additional tasks that you think necessary for this project)

1.	Build regression models to predict the price for second hand cars. This step should include EDA, pre-processing, creating model and evaluation. You may use several regression methods, do benchmarking, employ additional techniques, such as ensemble method, to improve the prediction accuracy, etc.
2.	Please use the most appropriate evaluation metrics for this regression project.

Deadline: 17 November 2020, 11:59 a.m. 
 
Deliverables: 
1. Jupyter notebook (your code)
2.	Presentation <br>
**Nb**: In the end of slide and report, please write members’ names and their own contribution during this project.

## Read dataset

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')

# Rename columns
df = df.rename(columns={
    "Kapasitas mesin":"kapasitas",
    "Tipe bodi":"bodi",
    "Fitur tambahan":"fitur",
    "Nama Bursa Mobil":"bursa",
    "Tipe bahan bakar":"fuel",
    "Tipe Penjual":"penjual",
    "Jarak tempuh":"jarak",
    "Sistem Penggerak":"penggerak",
})
for col in df.columns:
    df = df.rename(columns={
        col:col.lower()
    })

display(df.head())
print('Shape:', df.shape)
# credit to prev. year DSDM students: Felix Septianus Darmawan, Maulana Akmal, Steven Sukma limanus, Ricky Kennedy

,tahun,kapasitas,warna,bodi,varian,state,merek,transmisi,model,fitur,bursa,fuel,penjual,city,country,jarak,phone,penggerak,price,neighbourhood
0,2008,>1.500 - 2.000 cc,Abu-abu,Pick-up,G Luxury,Jawa Barat,Toyota,Automatic,Innova,"['Sensor Parkir', 'Electric Window']",NaN,Bensin,Individu,Bandung Kota,Indonesia,120.000-125.000,6.282282e+12,Rear Wheel Drive (RWD),125000000,Bojongloa Kidul
1,2018,>1.000 - 1.500 cc,Silver,Hatchback,E CVT,Jakarta D.K.I.,Honda,Automatic,HR-V,"['Airbag', 'Bluetooth Audio', 'Electric Window...",Bursa Mobil Blok M Square,Bensin,Diler,Jakarta Selatan,Indonesia,10.000-15.000,6.289655e+12,4X2,240000000,Kebayoran Baru
2,2017,>1.000 - 1.500 cc,Putih,Hatchback,Panca T,Sumatra Selatan,Datsun,Manual,Go,"['Airbag', 'Electric Window', 'Sensor Parkir']",NaN,Bensin,Diler,Palembang Kota,Indonesia,15.000-20.000,6.281279e+12,Front Wheel Drive (FWD),80000000,Kemuning
3,2013,>1.000 - 1.500 cc,Silver,NaN,E,Jawa Timur,Honda,Automatic,Freed,NaN,NaN,Bensin,NaN,Surabaya Kota,Indonesia,85.000-90.000,6.283832e+12,NaN,170000000,Gubeng
4,2016,>1.000 - 1.500 cc,Putih,MPV,GL Sporty,Jakarta D.K.I.,Suzuki,Manual,Ertiga,"['Airbag', 'Electric Window', 'Sensor Parkir']",Bursa Mobil MGK Kemayoran,Bensin,Diler,Jakarta Pusat,Indonesia,60.000-65.000,6.281912e+12,4X2,125000000,Kemayoran


Shape: (36575, 20)


## EDA & Preprocessing

In [4]:
# Copy df ke proc, proc yang dipake buat preprocess supaya masih ada data yg original
proc = df.copy()

In [19]:
df.describe()

,phone,price
count,3.404300e+04,3.657500e+04
mean,5.926282e+12,1.712408e+08
std,5.689054e+12,5.410271e+08
min,6.281110e+10,1.234500e+04
25%,6.281268e+12,8.500000e+07
50%,6.281703e+12,1.200000e+08
75%,6.285215e+12,1.747250e+08
max,6.289581e+13,8.975000e+10


In [31]:
df.isna().sum()

tahun                0
kapasitas         4383
warna                0
bodi              5589
varian            1586
state                0
merek                0
transmisi            0
model                0
fitur            11660
bursa            31616
fuel                 0
penjual           6434
city                 0
country              0
jarak                0
phone             2532
penggerak         7793
price                0
neighbourhood        2
dtype: int64

### Tahun

In [5]:
# Preprocess Tahun, ubah ke int
proc.tahun = proc.tahun.astype(str).str.replace("<1986", "1985")
proc.tahun = proc.tahun.astype(int)
proc.corr()

,tahun,phone,price
tahun,1.000000,-0.034172,0.116330
phone,-0.034172,1.000000,-0.019518
price,0.116330,-0.019518,1.000000


### Kapasitas mesin

In [6]:
# values
proc.kapasitas.value_counts()

>1.000 - 1.500 cc    15778
>1.500 - 2.000 cc     7220
>2.000 - 3.000 cc     6804
<1.000 cc             1396
>3.000 cc              994
Name: kapasitas, dtype: int64

In [7]:
# encode to mean of range
kapasitas_dict = {
    ">1.000 - 1.500 cc":    1250,
    ">1.500 - 2.000 cc":    1750,
    ">2.000 - 3.000 cc":    2500,
    "<1.000 cc":            500,
    ">3.000 cc":            3500,
}
kapasitas_mode = proc.kapasitas.mode()[0]

def map_kapasitas(x):
    if x is not np.nan:
        return kapasitas_dict[x]
    return kapasitas_dict[kapasitas_mode]

proc.kapasitas = proc.kapasitas.apply(map_kapasitas)

In [8]:
proc.corr()

,tahun,kapasitas,phone,price
tahun,1.000000,-0.134790,-0.034172,0.116330
kapasitas,-0.134790,1.000000,-0.034456,0.125535
phone,-0.034172,-0.034456,1.000000,-0.019518
price,0.116330,0.125535,-0.019518,1.000000


### Warna

In [9]:
proc.warna.value_counts(ascending=True)

Ungu         115
Emas         246
Oranye       275
Kuning       336
Marun        455
Coklat       620
Lainnya      675
Hijau        838
Biru        1909
Merah       2128
Abu-abu     4393
Silver      6832
Putih       7162
Hitam      10591
Name: warna, dtype: int64

In [10]:
# Frequency encoding
freq = proc.warna.value_counts(ascending=True)

proc.warna = proc.warna.apply(lambda x: freq.index.get_loc(x))

In [11]:
proc.corr()

,tahun,kapasitas,warna,phone,price
tahun,1.000000,-0.134790,0.232425,-0.034172,0.116330
kapasitas,-0.134790,1.000000,0.039759,-0.034456,0.125535
warna,0.232425,0.039759,1.000000,-0.012119,0.035529
phone,-0.034172,-0.034456,-0.012119,1.000000,-0.019518
price,0.116330,0.125535,0.035529,-0.019518,1.000000


### Price

In [12]:
proc.price.describe()

count    3.657500e+04
mean     1.712408e+08
std      5.410271e+08
min      1.234500e+04
25%      8.500000e+07
50%      1.200000e+08
75%      1.747250e+08
max      8.975000e+10
Name: price, dtype: float64

In [13]:
proc.sort_values(by="price", ascending=True).head()

,tahun,kapasitas,warna,bodi,varian,state,merek,transmisi,model,fitur,bursa,fuel,penjual,city,country,jarak,phone,penggerak,price,neighbourhood
19298,2004,2500,11,SUV,LGX-D,Jawa Timur,Toyota,Manual,Kijang,NaN,NaN,Diesel,Individu,Tuban Kab.,Indonesia,35.000-40.000,NaN,4X2,12345,Palang
16132,2010,2500,12,Double Cabin,HD-X,Kalimantan Timur,Mitsubishi,Manual,Strada Triton,NaN,NaN,Diesel,NaN,Balikpapan Kota,Indonesia,85.000-90.000,6.281258e+12,4X4,2500000,Balikpapan Selatan
29442,2019,1250,12,Minibus,S 1.5,Banten,Wuling,Manual,Confero S,NaN,NaN,Bensin,Individu,Tangerang Kota,Indonesia,0-5.000,6.285695e+12,NaN,7000000,Karawaci
2099,1985,1250,8,NaN,Lain-lain,Jawa Barat,Daihatsu,Manual,HIJET,NaN,NaN,Bensin,Individu,Bandung Kota,Indonesia,245.000-250.000,6.289665e+12,NaN,7000000,Sukajadi
27350,1985,500,9,Minibus,Lain-lain,Jawa Barat,Daihatsu,Manual,HIJET,NaN,NaN,Bensin,Individu,Bekasi Kab.,Indonesia,295.000-300.000,6.289507e+12,4X2,7500000,Sukakarya


In [14]:
# filter all dataset. Prices must be higher than threshold
price_lower_threshold = 7000000

proc = proc[proc.price >= price_lower_threshold]
proc.shape

(36573, 20)

In [15]:
# divide prices by 1e7
proc.price = proc.price / 1e7

### Find out data types

In [16]:
proc.dtypes

tahun              int32
kapasitas          int64
warna              int64
bodi              object
varian            object
state             object
merek             object
transmisi         object
model             object
fitur             object
bursa             object
fuel              object
penjual           object
city              object
country           object
jarak             object
phone            float64
penggerak         object
price            float64
neighbourhood     object
dtype: object

## Modelling

In [23]:
# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor,AdaBoostRegressor, VotingRegressor, BaggingRegressor
import xgboost as xgb
from sklearn import neighbors
from sklearn.svm import SVR
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Validation 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error

In [24]:
benchmark_models = [
      ('XGBRegressor', xgb.XGBRegressor(verbose=False)),
      ('CatBoostRegressor', CatBoostRegressor(verbose=False)),
      ('SVR', SVR()),
      ('LGBMRegressor', lgb.LGBMRegressor()),
      ('MLP', MLPRegressor()),
      ('KNeighborsRegressor', KNeighborsRegressor()),
      ('RandomForest ',RandomForestRegressor()),
      ('ExtraTreeRegressor :',ExtraTreesRegressor()),
      ('GradientBoostingRegressor', GradientBoostingRegressor()) ,
      ('AdaBoostRegressor', AdaBoostRegressor()),
      ('BaggingRegressor', BaggingRegressor()),
      ('LinearRegression', LinearRegression()),
      ('GaussianProcessRegressor', GaussianProcessRegressor())
    ]

def validate(X, y, n_splits, n_repeats, random_state, models):
    if (type(X) == pd.core.frame.DataFrame): X = X.values
    if (type(y) == pd.core.frame.DataFrame): y = y.values

    model_data = []
    for name,curr_model in models :
        curr_model_data = {}
        curr_model_data["Nama"] = name
        print(name)

        score_metrics = ['neg_mean_absolute_error', 'neg_root_mean_squared_error', 'neg_mean_squared_log_error']

        rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
        scores = cross_validate(curr_model, X, y, scoring=score_metrics,
        cv=rkf, return_train_score=True)

        for metric in score_metrics:
            curr_model_data["test_"+metric] = round(np.mean(scores["test_"+metric])*100,2)

        model_data.append(curr_model_data)

    return pd.DataFrame(model_data)

In [34]:
scores

{'fit_time': array([105.22597313, 113.60592151, 104.31695127, 106.07664585,
        110.9375031 , 110.99303317, 111.12536883, 115.64071512,
        100.71208668, 113.60521102, 103.636446  , 109.38645053,
        107.69192863, 114.00089216, 109.90922141]),
 'score_time': array([0.10000277, 0.10701585, 0.10000944, 0.10001135, 0.09699845,
        0.09700727, 0.09599972, 0.10200596, 0.08901501, 0.09399867,
        0.102005  , 0.11600614, 0.09599853, 0.11702752, 0.08400035]),
 'test_neg_mean_absolute_error': array([-5.06381479, -4.81908387, -4.34985692, -4.50559567, -5.02486845,
        -4.25224055, -5.95328635, -4.54445953, -5.21624469, -4.6254717 ,
        -4.61256808, -4.24572068, -4.8521938 , -4.56558746, -5.33190445]),
 'train_neg_mean_absolute_error': array([-5.28176704, -3.77215772, -4.47577519, -4.82089431, -3.78369208,
        -4.39384651, -5.47735178, -3.73310357, -5.179618  , -3.7333564 ,
        -4.46606289, -4.37564518, -5.18790086, -3.64066841, -5.30317276]),
 'test_neg_root_m

In [46]:
for key in scores:
    print(f"avg {key}: {scores[key].mean()}")

avg fit_time: 109.12428989410401
avg score_time: 0.09980680147806803
avg test_neg_mean_absolute_error: -4.797526466384711
avg train_neg_mean_absolute_error: -4.5083341803865356
avg test_neg_root_mean_squared_error: -40.16939780919828
avg train_neg_root_mean_squared_error: -42.71961563647137
